In [ ]:
from sklearn.metrics import f1_score , precision_score , recall_score , accuracy_score
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# data = pd.read_csv('data_test/trainNotNormalizedM1_15.csv', delimiter = ',').reset_index(drop=True)
# data = pd.read_csv('data_test/trainM1_15.csv', delimiter = ',').reset_index(drop=True)
data = pd.read_csv('data_test/M1-15+DemographicFeatureEng.csv', delimiter = ',').reset_index(drop=True)
data.shape

In [ ]:
# list(data)

In [ ]:
y_ans_data = pd.read_csv('data_test/M16All.csv', delimiter = ',')

In [ ]:
y_ans_data.shape

In [ ]:
x = data[['1','2','3','4','5','6','7','8','9','10','11','12','13','14','Age','AccAge','AgeWhenOpenAcc','mrch_cd_Indicator',
 'Age_Indicator',
 'AccAge_Indicator',
 'AgeWhenOpenAcc_Indicator',
 '15_Indicator',
 'New_cat_cd',
 'New_gnd_cd',
 'New_idv_incm_seg_cd',
 'New_ip_tp_cd',
 'New_ocp_cd',
 'New_mar_st_cd',
 'New_Province']]
y = data[['15']].values
x_ans = data[['2','3','4','5','6','7','8','9','10','11','12','13','14','15','Age','AccAge','AgeWhenOpenAcc','mrch_cd_Indicator',
 'Age_Indicator',
 'AccAge_Indicator',
 'AgeWhenOpenAcc_Indicator',
 '15_Indicator',
 'New_cat_cd',
 'New_gnd_cd',
 'New_idv_incm_seg_cd',
 'New_ip_tp_cd',
 'New_ocp_cd',
 'New_mar_st_cd',
 'New_Province']]
y_ans = y_ans_data[['Target']].values

In [ ]:
score1(y,y_ans)

In [ ]:
############################################
####### for transform index category #######
############################################
import Encoder as en
import numpy as np
listcate = ['New_cat_cd',
 'New_gnd_cd',
 'New_idv_incm_seg_cd',
 'New_ip_tp_cd',
 'New_ocp_cd',
 'New_mar_st_cd',
 'New_Province']
mcle = en.MultiColumnLabelEncoder(columns=np.array(listcate))
mcle.fit(x)
mcle.transform(x_ans)
mcle.transform(x)
x_ans = x_ans.values
x = x.values

print(x.shape,y.shape,x_ans.shape,y_ans.shape)

In [ ]:
print(x.shape,y.shape,x_ans.shape,y_ans.shape)

In [ ]:
from sklearn.model_selection import train_test_split
x, x_stack, y, y_stack = train_test_split(x, y, test_size=0.5, random_state=42)

In [ ]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import LabelEncoder
import numpy as np
x1 = pd.concat([x, x_ans])
x = x.to_dict('records')
x_ans = x_ans.to_dict('records')
x1 = x1.to_dict('records')
vec = DictVectorizer()

vec.fit(x1)
x = vec.transform(x).toarray()
x_ans = vec.transform(x_ans).toarray()

In [ ]:
from sklearn.decomposition import PCA , NMF
n_components = 15
# pca = PCA(n_components=n_components, svd_solver='full',random_state=42)
pca = NMF(n_components=n_components, init='random',random_state=42)
pca.fit(x)
x = pca.transform(x)
x_ans = pca.transform(x_ans)

In [ ]:
from sklearn import preprocessing
# x = preprocessing.normalize(x, norm='l2')MinMaxScaler
# x_ans = preprocessing.normalize(x_ans, norm='l2')
scaler = preprocessing.MinMaxScaler()
# scaler.fit(x[:,:22])
# x[:,:22] = scaler.transform(x[:,:22])
# x_ans[:,:22] = scaler.transform(x_ans[:,:22])

scaler.fit(x)
x = scaler.transform(x)
x_ans = scaler.transform(x_ans)

In [ ]:
from keras.models import Model
from keras.layers import Reshape,Bidirectional,Input, Dense,Embedding,GRU,Dropout,Conv1D,MaxPooling1D,Flatten,concatenate,Add,TimeDistributed,BatchNormalization
from keras.optimizers import Adam , RMSprop
from keras import regularizers
import keras.utils
from keras import optimizers
from keras.layers.normalization import BatchNormalization

def gru_model():
  input = Input(shape=(29,1))
  x = Dropout(0.10)(input)
#   x = input
  x = Conv1D(30,4, padding='same')(x)
  x = MaxPooling1D(pool_size=4, padding='same')(x)
#   x = BatchNormalization()(x)
  x = Conv1D(30,4, padding='same')(x)
  x = MaxPooling1D(pool_size=4, padding='same')(x)
#   x = BatchNormalization()(x)
#   x = Flatten()(x)
  x= Bidirectional(GRU(30))(x)
#   x = BatchNormalization()(x)
#   x = Reshape(14)(x)

  x = Dropout(0.15)(x)
#   x = Dense(5, activation='relu',kernel_regularizer=regularizers.l2(0.05))(x)
#   x = Bidirectional(GRU(20))(x)
#   x = Dropout(0.20)(x)
  out = Dense(1, activation='linear')(x)
  
#   sgd = optimizers.SGD(lr=1000)
  model = Model(inputs=input, outputs=[out])
  model.compile(optimizer='adam',
                loss='mean_absolute_error',
                metrics=['mae'])
  return model

In [ ]:
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import backend as K
K.clear_session()
weight_path_feedforward_nn='recomment.h5'
callbacks_list_feedforward_nn = [
#         TensorBoard(log_dir='/data/Graph/ff', histogram_freq=1, write_graph=True, write_grads=False),
        ModelCheckpoint(
            weight_path_feedforward_nn,
            save_best_only=True,
            save_weights_only=True,
            monitor='mae',
            mode='min',
            verbose=1
        )
  ]
print('start training GRU')
model_ = gru_model()
verbose = 1
n = 10000000
# n = 1
train_params = [(10 ,64)]
for (epochs, batch_size) in train_params:
  print("train with {} epochs and {} batch size".format(epochs, batch_size))
  model_.fit(np.reshape(x, (x.shape[0],x.shape[1],1)), y/n, epochs=epochs, batch_size=batch_size, verbose=verbose,
                           callbacks=callbacks_list_feedforward_nn,
                           validation_data=(np.reshape(x_ans, (x_ans.shape[0],x_ans.shape[1],1)), y_ans/n))

In [ ]:
y_pre = model_.predict(np.reshape(x_ans, (x_ans.shape[0],x_ans.shape[1],1)))
y_pre

In [ ]:
y_pre[y_pre < 0] = 0
score1(y_ans,y_pre*n)

In [ ]:
y_pre

In [ ]:
ycx_pre.shape

In [ ]:
state

In [ ]:
[(20 ,256)]

In [ ]:
np.reshape(x, (x.shape[0],x.shape[1],1)).shape

In [ ]:
from sklearn.metrics import mean_absolute_error , precision_score , recall_score , accuracy_score , roc_curve , auc , classification_report,fbeta_score
def smape(A, F):
#     F[F <= 0] = 0.01
#     F = np.exp(F)
    b = np.abs(A) + np.abs(F)
    b[b == 0] = 0.00001
    smape_s = 100/len(A) * np.sum(2 * np.abs(F - A) / (b))
    smape_s = 1-(smape_s/200)
    return smape_s

def score1(y_true,y_pre,thresholds = 0.5):
    mae = mean_absolute_error(y_true, y_pre)
#     mae = smape(y_true, y_pre)
#     print(mae,'smape')
    print(mae)
    

#     y_pre[y_pre >= threshold] = 1
#     y_pre[y_pre < threshold] = 0
#     print(fbeta_score(y_test, y_pre, pos_label=1 , average='binary', beta=0.5),'f2')

In [ ]:
train_params = [(20 ,256),(20 ,256)]
for (epochs, batch_size) in train_params:
  print("train with {} epochs and {} batch size".format(epochs, batch_size))

In [ ]:
from keras.models import Model
from keras.layers import Reshape,Bidirectional,Input, Dense,Embedding,GRU,Dropout,Conv1D,MaxPooling1D,Flatten,concatenate,Add,TimeDistributed,BatchNormalization
from keras.optimizers import Adam , RMSprop
from keras import regularizers
import keras.utils
from keras import optimizers
from keras.layers.normalization import BatchNormalization



def gru_model():
  input = Input(shape=(14,1))
  x = input
  x1 = Conv1D(30,4, padding='same')(x)
  x1 = MaxPooling1D(pool_size=4, padding='same')(x1)
  x1 = Conv1D(30,4, padding='same')(x1)
  x1 = MaxPooling1D(pool_size=4, padding='same')(x1)
  x1 = Flatten()(x1)
  x2 = Bidirectional(GRU(15))(x)
  x3 = Conv1D(30,4, padding='same')(x)
  x3 = MaxPooling1D(pool_size=4, padding='same')(x3)
  x3 = Conv1D(30,4, padding='same')(x3)
  x3 = MaxPooling1D(pool_size=4, padding='same')(x3)
  x3 = Bidirectional(GRU(15))(x3)
  x1 = BatchNormalization()(x1)
  x2 = BatchNormalization()(x2)
  x3 = BatchNormalization()(x3)
    
  x = Add()([x1,x2,x3])
  x = BatchNormalization()(x)
  x = Dropout(0.10)(x)
  x_ = x
  x = Reshape((30,1))(x)
  x1 = Conv1D(15,4, padding='same')(x)
  x1 = MaxPooling1D(pool_size=4, padding='same')(x1)
  x1 = Conv1D(15,4, padding='same')(x1)
  x1 = MaxPooling1D(pool_size=4, padding='same')(x1)
  x1 = Flatten()(x1)

  x2 = Bidirectional(GRU(15))(x)
  x3 = Conv1D(20,4, padding='same')(x)
  x3 = MaxPooling1D(pool_size=4, padding='same')(x3)
  x3 = Conv1D(20,4, padding='same')(x3)
  x3 = MaxPooling1D(pool_size=4, padding='same')(x3)
#   x1 = Flatten()(x1)
  x3 = Bidirectional(GRU(15))(x3)
  x1 = BatchNormalization()(x1)
  x2 = BatchNormalization()(x2)
  x3 = BatchNormalization()(x3)
    
  x = Add()([x1,x2,x3,x_])
  x = BatchNormalization()(x)

  x = Dropout(0.10)(x)
  x_2 = x
  x = Reshape((30,1))(x)
  x1 = Conv1D(15,4, padding='same')(x)
  x1 = MaxPooling1D(pool_size=4, padding='same')(x1)
  x1 = Conv1D(15,4, padding='same')(x1)
  x1 = MaxPooling1D(pool_size=4, padding='same')(x1)
  x1 = Flatten()(x1)

  x2 = Bidirectional(GRU(15))(x)
  x3 = Conv1D(20,4, padding='same')(x)
  x3 = MaxPooling1D(pool_size=4, padding='same')(x3)
  x3 = Conv1D(20,4, padding='same')(x3)
  x3 = MaxPooling1D(pool_size=4, padding='same')(x3)
  x3 = Bidirectional(GRU(15))(x3)
  x1 = BatchNormalization()(x1)
  x2 = BatchNormalization()(x2)
  x3 = BatchNormalization()(x3)

  x = Add()([x1,x2,x3,x_,x_2])
#   x = BatchNormalization()(x)
#   x = Dropout(0.15)(x)
  out = Dense(1, activation='linear')(x)
  
#   sgd = optimizers.SGD(lr=1000)
  model = Model(inputs=input, outputs=out)
  model.compile(optimizer='adam',
                loss='mean_absolute_error',
                metrics=['mae'])
  return model

In [ ]:
yc = np.copy(y)
yc[yc != 0] = 1
yc_ans = np.copy(y_ans)
yc_ans[yc_ans != 0] = 1
# yc_ans

In [ ]:
from imblearn.over_sampling import SMOTE, ADASYN  
from imblearn.combine import SMOTEENN ,SMOTETomek

sm = SMOTEENN(random_state=42)
# sm = SMOTETomek(random_state=42)
# sm = ADASYN(random_state=42)
# sm.fit(x)
# x_sm, y_sm = sm.fit_sample(x_train, y_train)
x, yc = sm.fit_sample(x, yc)

print(x.shape,y.shape)

In [ ]:
############################################
############### Xgboost ####################
############################################
import xgboost as xgb
# dsm = xgb.DMatrix(x, label=y)
dt = xgb.DMatrix(x_ans, label=yc_ans)
dx = xgb.DMatrix(x, label=yc)
                  
# dt = xgb.DMatrix(x_ans_, label=yc_ans)
# dx = xgb.DMatrix(x_, label=yc)

# dsm = xgb.DMatrix(x_sm, label=y_sm)                  
evallist = [(dt, 'eval'), (dx, 'train')]

num_round = 10000
# binary:logistic
# param = {
#  'objective': 'reg:linear',
#  'colsample_bytree': 0.9683760122352089,
#  'gamma':0.8835260600913024,
#  'learning_rate': 0.199426498504554,
#  'max_depth': 20,
#  'min_child_weight': 10.95324500379702,
#  'n_estimators': 25,
# #  'objective': 'binary:logistic',
#  'scale_pos_weight': 1,
#  'seed': 42,
#  'eval_metric': 'mae',
#  'lambda': 150,
#  'alpha': 100,
# #  'rate_drop':0.950292864879127905,
# #  'tree_method':'exact',
#  'normalize_type':'forest',
#  'subsample': 0.9035691355661921}gblinear

param = {
#     'booster': 'dart',
#     'booster': 'gblinear',
  'objective': 'binary:logistic',
 'colsample_bytree': 0.9683760122352089,
 'gamma':0.8835260600913024,
#  'learning_rate': 0.1249426498504554,
#  'max_depth': 20,
 'min_child_weight': 110.95324500379702,
 'n_estimators': 19,
 'scale_pos_weight': 1,
 'seed': 42,
 'eval_metric': 'auc',
 'lambda': 20,
 'alpha': 120,
# 'updater':'coord_descent',
#     'feature_selector':'greedy',
#     'top_k':12,
 'rate_drop':0.950292864879127905,
 'tree_method':'exact',
 'normalize_type':'forest',
 'subsample': 0.9035691355661921,
 'skip_drop' : 0.6,
 'one_drop': False,
}
 
evals_result = {}

In [ ]:
bst = xgb.train(param, dx, num_round, evallist, evals_result=evals_result,early_stopping_rounds=50)

In [ ]:
treee = bst.best_ntree_limit
# treee = 20
# yc_pre = np.array(bst.predict(dt, ntree_limit=treee))
yc_pre = bst.predict(dt)
yc_pre


In [ ]:
threshold = 0.5910990907732
threshold = 0.3972261
ycx_pre  = yc_pre > threshold
ycx_pre = ycx_pre.astype(int)
f1_score(yc_ans, ycx_pre, average='binary',pos_label=0)

In [ ]:
from sklearn.metrics import precision_score,precision_recall_fscore_support
fpr, tpr, thresholds = roc_curve(yc_ans, yc_pre, pos_label=1)
for i in thresholds:
#     print(i)
    ycx_pre  = yc_pre > i
    ycx_pre = ycx_pre.astype(int)
#     print(f1_score(yc_ans, ycx_pre, average='binary',pos_label=1),i)
    print(precision_score(yc_ans, ycx_pre, average='micro',pos_label=1),i)

In [ ]:
fpr, tpr, thresholds = roc_curve(yc_ans, yc_pre, pos_label=1)
auc(fpr, tpr)
# mean_absolute_error(yc_ans, yc_pre)

In [ ]:
xgb.plot_importance(bst,max_num_features =20)

In [ ]:
yc_pre

In [ ]:
from sklearn.naive_bayes import BernoulliNB
clf = BernoulliNB()
clf.fit(x, yc)

In [ ]:
clf.predict_proba(x_ans)

In [ ]:
yc_pre = clf.predict_proba(x_ans)[:,0]
yc_pre = clf.predict(x_ans)
# >>> fpr, tpr, thresholds = metrics.roc_curve(y, pred, pos_label=2)
# >>> metrics.auc(fpr, tpr)

ycx_pre  = yc_pre > threshold
ycx_pre = ycx_pre.astype(int)
f1_score(yc_ans, ycx_pre, average='binary',pos_label=0)

In [ ]:
yc_pre

In [ ]:
gbr = GradientBoostingRegressor(n_estimators=80, max_depth=18,
                                    learning_rate=0.4, loss='lad',
                                    random_state=42)

In [ ]:
import numpy as np
# import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble.partial_dependence import plot_partial_dependence
from sklearn.ensemble.partial_dependence import partial_dependence
from sklearn import linear_model
from sklearn import grid_search
import scipy.stats as st
from sklearn.model_selection import RandomizedSearchCV , GridSearchCV
import os
import numpy as np
import datetime

import scipy.stats as st

from sklearn.svm import SVR
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.ensemble import ExtraTreesRegressor, VotingClassifier
from sklearn.model_selection import RandomizedSearchCV , GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import LabelBinarizer
from xgboost import XGBRegressor
from sklearn.externals import joblib
import catboost as cb
from lightgbm.sklearn import LGBMRegressor
from sklearn.ensemble import BaggingRegressor


gbr = GradientBoostingRegressor(n_estimators=80, max_depth=18,
                                    learning_rate=0.4, loss='lad',
                                    random_state=42)
elas = linear_model.ElasticNetCV(cv=5, random_state=0,l1_ratio=[0.05,.1,0.15,.2, .5, .9,0.95, 1],n_jobs=-1
                                 ,normalize=False, positive=False)

n_jobs = 3
# n_iter = 20
# n_iter_nt = 1
# n_components = 25
cv = 5
seed=42
# n_features= 25
n_features= 10
scoring = 'neg_mean_absolute_error'
scoring = 'neg_mean_squared_log_error'
# scoring = 'r2'
estimator = RandomForestRegressor()
params = {
    "max_depth": [1, 2, 3,5,10,20,30,40,50,100, None],
    "max_features": st.randint(1, 300),
#     "max_features": ["log2"],
    "min_samples_split": st.randint(2, 10),
#     "min_samples_leaf": st.randint(1, n_features),
#     "bootstrap": [True, False],
    "criterion": ["gini", "entropy"],
    'oob_score': [True, False],
    'random_state': [seed],
    'warm_start' : [True],
}
rnf = RandomizedSearchCV(estimator, params, cv=cv,n_jobs=n_jobs, n_iter=10, scoring = scoring) 
clf = gbr

bag = BaggingRegressor(elas,n_jobs = n_jobs, n_estimators=30,warm_start =True,bootstrap_features =True)
clf = bag
# clf = elas
clf = rnf
# estimator = SVR()
# params = {  
#     'C':[0.001, 0.01, 0.1, 1, 10], 
#     'degree': st.randint(1, 10),
#     'shrinking': [True, False],
# #     'probability': [True],
#     'tol': [1e-3],
# #     'random_state': [seed],
# }
# clf = RandomizedSearchCV(estimator, params, cv=cv,n_jobs=n_jobs, n_iter=10, scoring = 'neg_mean_absolute_error')


clf.fit(x, y)
# y_a = clf.predict(x_test)

In [ ]:
y_clf = clf.predict(x_ans)
y_clf[y_clf < 0] = 0
y_clf[y_clf > 2952646748] = 2952646748
# y_ans = y_ans[:,0]
score1(y_ans,y_clf)
score1(y_ans,y_clf*yc_pre)
score1(y_ans,y_clf*ycx_pre)

In [ ]:
clf.predict(x_ans)

In [ ]:
param = {
 'objective': 'reg:linear',
 'colsample_bytree': 0.9683760122352089,
 'gamma':0.8835260600913024,
 'learning_rate': 0.199426498504554,
 'max_depth': 20,
 'min_child_weight': 10.95324500379702,
 'n_estimators': 25,
 'scale_pos_weight': 12,
 'seed': 42,
 'eval_metric': 'mae',
 'lambda': 150,
 'alpha': 100,
 'rate_drop':0.50292864879127905,
 'tree_method':'exact',
 'normalize_type':'forest',
 'subsample': 0.9035691355661921
}

clf = XGBRegressor(**param)
clf.fit(x,y)
y_clf = clf.predict(x_ans)
y_clf[y_clf < 0] = 0
y_clf[y_clf > 2952646748] = 2952646748
# y_ans = y_ans[:,0]
score1(y_ans,y_clf)
score1(y_ans,y_clf*yc_pre)
score1(y_ans,y_clf*ycx_pre)

In [ ]:
xgb2 = XGBRegressor(**param).fit(x,y)
rnf2 = rnf.fit(x,y)
a = xgb2.predict(x_stack)
b = rnf2.predict(x_stack)
x_2 = np.column_stack((a,b))
a = xgb2.predict(x_ans)
b = rnf2.predict(x_ans)
x_ans_2 = np.column_stack((a,b))

x_2

In [ ]:
estimators = [("xbg", XGBRegressor(**param)),
              ("rnf", rnf),
              ("elas", elas),]

In [ ]:
np.zeros((x_stack.shape[0], len(estimators))).shape

In [ ]:
x_2 = np.zeros((x_stack.shape[0], len(estimators)))
x_ans_2 = np.zeros((x_ans.shape[0], len(estimators)))
for n, (name, estimator) in enumerate(estimators):
#     print(n, name, estimator)
    y_estimator = estimator.fit(x,y).predict(x_stack)
    x_2[:,n] = y_estimator
    x_ans_2[:,n] = estimator.predict(x_ans)
    score1(y_stack,y_estimator)
    print(name)
x_2

In [ ]:
clf

In [ ]:
clf.predict(x_ans)

In [ ]:
clf.coef_

In [ ]:
from matplotlib import pyplot
# a = clf.coef_
# a = clf.best_estimator_.feature_importances_
a = clf.feature_importances_
# a = a[:15]
pyplot.bar(range(len(a)), a)
pyplot.show()

In [ ]:
pyplot.show()

In [ ]:
clf.best_estimator_.feature_importances_

In [ ]:
clf.best_estimator_

In [ ]:
features = list(range(1,15))
features = [(14,15),15]
fig, axs = plot_partial_dependence(clf, x_ans, features,
#                                        feature_names=list(data1)[:10],
                                       n_jobs=-1, grid_resolution=1000)

In [ ]:
############################################
############### Xgboost ####################
############################################
import xgboost as xgb
# dsm = xgb.DMatrix(x, label=y)
dt = xgb.DMatrix(x_ans_2, label=y_ans)
dx = xgb.DMatrix(x_2, label=y_stack)
                  
# dt = xgb.DMatrix(x_ans_, label=yc_ans)
# dx = xgb.DMatrix(x_, label=yc)

# dsm = xgb.DMatrix(x_sm, label=y_sm)                  
evallist = [(dt, 'eval'), (dx, 'train')]

num_round = 100

param = {
    'booster':'gblinear',
 'objective': 'reg:linear',
 'colsample_bytree': 0.9683760122352089,
 'gamma':0.8835260600913024,
#  'learning_rate': 0.199426498504554,
 'max_depth': 20,
 'min_child_weight': 10.95324500379702,
 'n_estimators': 25,
#  'objective': 'binary:logistic',
 'scale_pos_weight': 1,
 'seed': 42,
 'eval_metric': 'mae',
 'lambda': 150,
 'alpha': 100,
#  'rate_drop':0.50292864879127905,
 'tree_method':'exact',
 'normalize_type':'forest',
 'subsample': 0.9035691355661921
}
# gblinear

 
evals_result = {}
bst = xgb.train(param, dx, num_round, evallist, evals_result=evals_result,early_stopping_rounds=50)

In [ ]:
treee = bst.best_ntree_limit
# treee = 20
# y_clf = np.array(bst.predict(dt, ntree_limit=treee))
y_clf = bst.predict(dt)
y_clf[y_clf < 0] = 0
y_clf[y_clf > 2952646748] = 2952646748
# y_ans = y_ans[:,0]
score1(y_ans,y_clf)
score1(y_ans,y_clf*yc_pre)
score1(y_ans,y_clf*ycx_pre)

In [ ]:
clf = bag
clf.fit(x_2,y_stack)
y_clf = clf.predict(x_ans_2)
y_clf[y_clf < 0] = 0
y_clf[y_clf > 2952646748] = 2952646748
# y_ans = y_ans[:,0]
score1(y_ans,y_clf)
score1(y_ans,y_clf*yc_pre)
score1(y_ans,y_clf*ycx_pre)

In [ ]:
a = np.array([])
# a[0] = 0

In [ ]:
a

In [ ]:
# a = clf.coef_
a = clf.best_estimator_.feature_importances_
n = 0.01
a[a > n]


In [ ]:
np.argwhere(a != 0)[:,0]
x_ = x[:,np.argwhere(a > n)[:,0]]
x_ans_ = x_ans[:,np.argwhere(a > n)[:,0]]

In [ ]:
x_ = x[:,:14]
x_ans_ = x_ans[:,:14]

In [ ]:
x_.shape

In [ ]:
clf_ = linear_model.ElasticNetCV(cv=5, random_state=0,l1_ratio=[0.05,.1,0.15,.2, .5, .9,0.95, 1],n_jobs=-1
                                 ,normalize=False, positive=False)
n_features= x_.shape[1]
scoring = 'neg_mean_absolute_error'
# scoring = 'neg_mean_squared_error'

estimator = RandomForestRegressor()

# Parameter for RandomForestClassifier
params = {
    "max_depth": [3, None],
    "max_features": st.randint(1, n_features),
    "min_samples_split": st.randint(2, 10),
    "min_samples_leaf": st.randint(1, n_features),
#     "bootstrap": [True, False],
    "criterion": ["mse"],
    'oob_score': [True, False],
    'random_state': [seed],
}
rnf = RandomizedSearchCV(estimator, params, cv=cv,n_jobs=n_jobs, n_iter=50, scoring = scoring) 

clf_ = gbr
# clf_ = rnf
clf_.fit(x_, y)

In [ ]:
y_clf = clf_.predict(x_ans_)
y_clf[y_clf < 0] = 0
y_clf[y_clf > 2952646748] = 2952646748
# y_ans = y_ans[:,0]
# y_clf = y_clf[:,0]
score1(y_ans,y_clf)
score1(y_ans,y_clf*yc_pre)
score1(y_ans,y_clf*ycx_pre)

In [ ]:
len(x_[0])

In [ ]:
features = list(range(0,10))
fig, axs = plot_partial_dependence(clf_, x_ans_, features,
#                                        feature_names=list(data1)[:10],
                                       n_jobs=-1, grid_resolution=100)

In [ ]:
y_clf

In [ ]:
########### models ################ param
############################################
######## scikit-learn multi-models #########
############################################
import os
import numpy as np
import datetime

import scipy.stats as st

from sklearn.svm import SVR
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.ensemble import ExtraTreesRegressor, VotingClassifier
from sklearn.model_selection import RandomizedSearchCV , GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import LabelBinarizer
from xgboost import XGBRegressor
from sklearn.externals import joblib
import catboost as cb
from lightgbm.sklearn import LGBMRegressor

n_jobs = 3
n_iter = 30
n_iter_nt = 3
n_components = 25
cv = 5
seed=42
n_features= 14
# n_features= x_sm.shape[1]
# n_features= n_components
is_pca = False
scoring = 'neg_mean_absolute_error'
# is_pca = True

# estimator = XGBClassifier(nthreads=-1,tree_method='exact')
param = {'objective': 'reg:linear',
 'colsample_bytree': 0.9683760122352089,
 'gamma': 0.7790711924812199,
 'learning_rate': 0.2249426498504554,
 'max_depth': 20,
 'min_child_weight': 10.95324500379702,
 'n_estimators': 150,
 'scale_pos_weight': 1,
 'seed': 42,
 'eval_metric': ['mae'],
 'lambda': 2,
 'alpha': 15,
 'rate_drop':0.1,
 'tree_method':'exact',
 'normalize_type':'forest',
 'subsample': 0.9035691355661921}

estimator = XGBRegressor(**param)
# objective = 'binary:logistic'

# Parameter for XGBoost
params = {  
    "n_estimators": st.randint(3, 40),
    "max_depth": st.randint(3, 30),
    "learning_rate": st.uniform(0.05, 0.4),
    "colsample_bytree": st.beta(10, 1),
    "subsample": st.beta(10, 1),
    "gamma": st.uniform(0, 10),
#     'objective': [objective],
    'scale_pos_weight': st.randint(0, 2),
    "min_child_weight": st.expon(0, 50),
    'lambda': st.randint(1, 20),
    'alpha': st.randint(0, 20),
    'rate_drop':st.uniform(0, 1),
    "seed": [seed],
}

xgb = RandomizedSearchCV(estimator, params, cv=cv,n_jobs=n_jobs, n_iter=n_iter, scoring = scoring) 
if(is_pca):
    estimators = [('reduce_dim', PCA(n_components=n_components, svd_solver='full')), ('clf', xgb)]
    xgb = Pipeline(estimators)
    xgb = GridSearchCV(xgb, cv=5, n_jobs=-1, param_grid=param_grid)
    
#################################################################
#################################################################

estimator = LogisticRegression()
# Parameter for LogisticRegression
params = {
    "penalty": ['l2'],
    "C": [0.001, 0.01, 0.1, 1, 10],
    "tol": [1e-4, 1e-3, 1e-2, 1e-1],
    "solver": ['newton-cg', 'lbfgs', 'liblinear', 'sag'],
    "max_iter": st.randint(50, 100),
    'random_state': [seed],
} 
log = RandomizedSearchCV(estimator, params, cv=cv,n_jobs=n_jobs, n_iter=n_iter , scoring = scoring) 
if(is_pca):
    estimators = [('reduce_dim', PCA(n_components=n_components, svd_solver='full')), ('clf', log)]
    log = Pipeline(estimators)
    log = GridSearchCV(log, cv=5, n_jobs=-1, param_grid=param_grid)

#################################################################
#################################################################

estimator = KNeighborsRegressor()
# Parameter for KNeighborsClassifier
params = {
    "n_neighbors": st.randint(2, 50),
    "weights": ['uniform', 'distance'],
    "algorithm": ['ball_tree', 'kd_tree', 'brute'],
    "leaf_size": st.randint(10, 30),
    "p": st.randint(1, 2),
}
knn = RandomizedSearchCV(estimator, params, cv=cv,n_jobs=n_jobs, n_iter=n_iter_nt , scoring = scoring)
if(is_pca):
    estimators = [('reduce_dim', PCA(n_components=n_components, svd_solver='full')), ('clf', knn)]
    knn = Pipeline(estimators)
    
#################################################################
#################################################################

estimator = RandomForestRegressor()

# Parameter for RandomForestClassifier
params = {
    "max_depth": [3, None],
    "max_features": st.randint(1, n_features),
    "min_samples_split": st.randint(2, 10),
    "min_samples_leaf": st.randint(1, n_features),
    "bootstrap": [True, False],
    "criterion": ["mse"],
    'random_state': [seed],
}
rnf = RandomizedSearchCV(estimator, params, cv=cv,n_jobs=n_jobs, n_iter=n_iter, scoring = scoring) 
if(is_pca):
    estimators = [('reduce_dim', PCA(n_components=n_components, svd_solver='full')), ('clf', rnf)]
    rnf = Pipeline(estimators)
    rnf = GridSearchCV(rnf, cv=5, n_jobs=-1, param_grid=param_grid)
    
#################################################################
#################################################################

estimator = ExtraTreesRegressor()
# 
# Parameter for ExtraTreesClassifier
params = {
    "n_estimators": st.randint(5, 50),
    "max_depth": [3, None],
    "max_features": st.randint(1, n_features),
    "min_samples_split": st.randint(2, 10),
    "min_samples_leaf": st.randint(1, n_features),
    "bootstrap": [True],
    "oob_score": [False,True],
#     "criterion": ["gini", "entropy"],
    'random_state': [seed],
    
}
ext = RandomizedSearchCV(estimator, params, cv=cv,n_jobs=n_jobs, n_iter=n_iter, scoring = scoring) 
if(is_pca):
    estimators = [('reduce_dim', PCA(n_components=n_components, svd_solver='full')), ('clf', ext)]
    ext = Pipeline(estimators)
    ext = GridSearchCV(ext, cv=5, n_jobs=-1, param_grid=param_grid)

#################################################################
#################################################################

estimator = AdaBoostRegressor()
# Parameter for AdaBoost
params = { 
    'n_estimators':st.randint(10, 100), 
    'learning_rate':st.beta(10, 1), 
#     'algorithm':['SAMME', 'SAMME.R'],
    'random_state': [seed],
}
ada = RandomizedSearchCV(estimator, params, cv=cv,n_jobs=n_jobs, n_iter=n_iter, scoring = scoring) 
if(is_pca):
    estimators = [('reduce_dim', PCA(n_components=n_components, svd_solver='full')), ('clf', ada)]
    ada = Pipeline(estimators)
    ada = GridSearchCV(ada, cv=5, n_jobs=-1, param_grid=param_grid)
    


#################################################################
#################################################################
estimator = SVR()
# Parameter for SVC
params = {  
    'C':[0.001, 0.01, 0.1, 1, 10], 
    'degree': st.randint(1, 10),
    'shrinking': [True, False],
#     'probability': [True],
    'tol': [1e-3],
#     'random_state': [seed],
}
svc = RandomizedSearchCV(estimator, params, cv=cv,n_jobs=n_jobs, n_iter=n_iter_nt, scoring = scoring)
if(is_pca):
    estimators = [('reduce_dim', PCA(n_components=n_components, svd_solver='full')), ('clf', svc)]
    svc = Pipeline(estimators)
#     svc = GridSearchCV(svc, cv=5, n_jobs=-1, param_grid=param_grid)

#################################################################
#################################################################

# Parameter for LGBMClassifier
params = {'boosting_type': 'gbdt',
          'max_depth' : -1,
#           'objective': 'binary',
          'nthread': 3, # Updated from nthread
          'num_leaves': 64,
          'learning_rate': 0.05,
          'max_bin': 512,
          'subsample_for_bin': 200,
          'subsample': 1,
          'subsample_freq': 1,
          'colsample_bytree': 0.8,
          'reg_alpha': 5,
          'reg_lambda': 10,
          'min_split_gain': 0.5,
          'min_child_weight': 1,
          'min_child_samples': 5,
          'scale_pos_weight': 1,
          'num_class' : 1,
          'metric' : 'mape'}

gridParams = {
    'learning_rate': [0.005],
    'n_estimators': [40],
    'num_leaves': [6,8,12,16],
    'boosting_type' : ['gbdt'],
    'objective' : ['mape'],
    'random_state' : [42,502], # Updated from 'seed'
    'colsample_bytree' : [0.65, 0.66],
    'subsample' : [0.7,0.75],
    'reg_alpha' : [1,1.2],
    'reg_lambda' : [1,1.2,1.4],
    }

estimator = LGBMRegressor(boosting_type= 'gbdt',
          objective = 'mape',
          n_jobs = 3, # Updated from 'nthread'
          silent = True,
          max_depth = params['max_depth'],
          max_bin = params['max_bin'],
          subsample_for_bin = params['subsample_for_bin'],
          subsample = params['subsample'],
          subsample_freq = params['subsample_freq'],
          min_split_gain = params['min_split_gain'],
          min_child_weight = params['min_child_weight'],
          min_child_samples = params['min_child_samples'],
          scale_pos_weight = params['scale_pos_weight'])

lgb = GridSearchCV(estimator, gridParams, verbose=0, cv=cv,n_jobs=n_jobs , scoring = scoring)
if(is_pca):
    print('asas')
    estimators = [('reduce_dim', PCA(n_components=n_components, svd_solver='full')), ('clf', lgb)]
    lgb = Pipeline(estimators)
    lgb = GridSearchCV(lgb, cv=5, n_jobs=-1, param_grid=param_grid)
# lgb = RandomizedSearchCV(estimator, params, cv=cv,n_jobs=n_jobs, n_iter=n_iter_nt)

#################################################################
#################################################################

estimator = cb.CatBoostRegressor()
params = {'depth': st.randint(3, 16),
          'learning_rate' : st.uniform(0.05, 0.4),
         'l2_leaf_reg': st.randint(0, 10),
         'iterations': [1]}

cat = RandomizedSearchCV(estimator, params, cv=cv,n_jobs=n_jobs, n_iter=2, scoring = scoring) 

In [ ]:
from sklearn.decomposition import PCA ,NMF
from sklearn.pipeline import Pipeline
nmf = PCA(n_components=13, random_state=42)
clf = svc
estimators = [('reduce_dim',nmf ), ('model', clf)]
# clf = Pipeline(estimators)

clf.fit(x, y)
y_clf = clf.predict(x_ans)
y_clf[y_clf < 0] = 0
y_clf[y_clf > 2952646748] = 2952646748
score1(y_ans,y_clf)
score1(y_ans,y_clf*yc_pre)
score1(y_ans,y_clf*ycx_pre)

In [ ]:
yc_pre.shape

In [ ]:
score1(y_ans,y_clf)
score1(y_ans,y_clf[:,0]*yc_pre)
score1(y_ans,y_clf[:,0]*ycx_pre)